In [1]:
import numpy as np
import torch
from args import parse_args
from graphimb import *
from data_utils import (
    get_data_stat,
    get_natural_imbalanced_split_data,
    get_step_imbalanced_split_data,
    load_data, 
)
from tobe import TopoBalanceAugmenter
from trainer import NodeClassificationTrainer
from utils import get_model, get_device, print_centered

# Running Experiment

This script will run experiments on all the combinations of following settings (specified by the global variables below):

| Setting         | (Default) Values                           | Description                                                        |
| --------------- | ------------------------------------------ | ------------------------------------------------------------------ |
| `DATASET_SPACE` | `['cora', 'citeseer', 'pubmed']`           | The datasets to use.                                               |
| `IMB_TYPES`     | `{'step': [10, 20], 'natural': [50, 100]}` | The imbalance types and ratios.                                    |
| `TOBE_MODES`    | `['dummy', 'pred', 'topo']`                | The ToBE modes to test, `dummy` means no topological augmentation. |

For other settings, we use the default values specified in `config.yaml`.

In [2]:
""" Experiment Setup """

import sys
sys.argv = [""]
args = parse_args()

DATASET_SPACE = ["cora", "citeseer", "pubmed"]
MODE_SPACE = ["dummy", "pred", "topo"]
IMB_SPACE = {
    'step': [10, 20],
    'natural': [50, 100],
}

In [3]:
args.gpu_id = 1

device = get_device(args.gpu_id)

print(
    f"Run experiment with\n"
    f"  - Datasets:        {DATASET_SPACE}\n"
    f"  - ToBE modes:      {MODE_SPACE}\n"
    f"  - Imbalance types: {IMB_SPACE}\n"
)

print_centered("Arguments", 40, fillchar="=")
kwlen = max([len(k) for k in args.__dict__.keys()]) + 1
for keys, values in args.__dict__.items():
    print(f"{keys:{kwlen}}: {values}")
print_centered("", 40, fillchar="=")

# run the experiment

for imb_type in IMB_SPACE.keys():  # loop over imbalance types
    
    for imb_ratio in IMB_SPACE[imb_type]: # loop over imbalance ratios
    
        print_centered(f"Imbalance Type [{imb_type.title()}] - Ratio [{imb_ratio}]", 80, fillchar="/", prefix="\n")
        
        for dataset in DATASET_SPACE:  # loop over datasets

            print_centered(f"Dataset [{dataset.title()}]", 80, fillchar="=", prefix="\n")

            args.imb_type = imb_type
            args.imb_ratio = imb_ratio
            args.dataset = dataset
            
            for tobe_mode in MODE_SPACE:    # loop over ToBE modes
                
                print_centered(f"Dataset [{args.dataset.title()}] - {args.imb_type.title()}IR [{args.imb_ratio}] - ToBE [{tobe_mode}]", 80, fillchar="=")

                best_results = []
                for i_run in range(1, args.n_runs + 1):
                    seed = args.seed + i_run

                    # load imbalanced data
                    data = load_data(args.dataset, to_device=device, verbose=args.debug)
                    if args.imb_type == "step":
                        data = get_step_imbalanced_split_data(
                            data, imbratio=args.imb_ratio, random_seed=seed, verbose=args.debug
                        )
                    elif args.imb_type == "natural":
                        data = get_natural_imbalanced_split_data(
                            data, imbratio=args.imb_ratio, random_seed=seed, verbose=args.debug
                        )
                    else:
                        raise ValueError(
                            f"imb_type must be one of ['step', 'natural'], got {args.imb_type}."
                        )
                    data = get_data_stat(data, store_in_data=True, verbose=args.debug)

                    # initialize model
                    model = get_model(
                        gnn_arch=args.gnn_arch,
                        feat_dim=data.n_feat,
                        hid_dim=args.hid_dim,
                        out_dim=data.n_class,
                        n_layer=args.n_layer,
                        device=device,
                    )
                    # tobe augmenter
                    augmenter = TopoBalanceAugmenter(mode=tobe_mode, random_state=seed)
                    # trainer
                    trainer = NodeClassificationTrainer(
                        model=model,
                        data=data,
                        device=device,
                        augmenter=augmenter,  # ToBE augmentation, to disable, set augmenter=None
                        learning_rate=args.lr,
                        weight_decay=args.weight_decay,
                        train_epoch=args.epochs,
                        early_stop_patience=args.early_stop,
                        eval_freq=1,
                        verbose_freq=None,
                        enable_tqdm=args.tqdm,
                        random_state=seed,
                    )
                    # train the GNN with ToBE augmentation
                    best_model = trainer.train()
                    # print best results
                    trainer.print_best_results()
                    # save best results
                    best_results.append(trainer.best_eval_results)

                # print the average performance of the best model
                info = f"Avg Test Performance ({args.n_runs} runs): "
                for metric in trainer.eval_metrics.keys():
                    scores = np.array(
                        [
                            best_results[i][metric]["test"] * 100
                            for i in range(len(best_results))
                        ]
                    )
                    info += f" | {metric.upper()}: {scores.mean():.2f} ± {scores.std()/(len(scores)**0.5):.2f}"
                print(info)

Now using GPU #1: Tesla V100-SXM2-32GB
Run experiment with
  - Datasets:        ['cora', 'citeseer', 'pubmed']
  - ToBE modes:      ['dummy', 'pred', 'topo']
  - Imbalance types: {'step': [10, 20], 'natural': [50, 100]}

============== Arguments ==============
gpu_id       : 1
seed         : 42
n_runs       : 5
debug        : False
dataset      : cora
imb_type     : step
imb_ratio    : 10
gnn_arch     : GCN
n_layer      : 3
hid_dim      : 256
lr           : 0.01
weight_decay : 0.0005
epochs       : 2000
early_stop   : 200
tqdm         : False
tobe_mode    : all

////////////////////// Imbalance Type [Step] - Ratio [10] //////////////////////

================================ Dataset [Cora] ================================
================= Dataset [Cora] - StepIR [10] - ToBE [dummy] =================
Best Epoch:   97 | train/val/test | BACC: 100.0/61.93/60.55 | MACRO-F1: 100.0/59.65/59.29 | node/edge ratio: 100.00/100.00% 
Best Epoch:  122 | train/val/test | BACC: 100.0/59.97/59.05 | M